In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from IPython.display import display, Image
from scipy import stats
from sklearn.decomposition import PCA
import statsmodels.api as sm


In [2]:
#rnaseq = pd.read_csv('mTEC_RNAseq_results_all.csv')
rnaseq = pd.read_pickle('common_data/Kallisto_abundance.pkl')
#tss=pd.read_pickle('df_all_new.pkl')
tss=pd.read_pickle('common_data/fivepseq_abundance_tsr.pkl')

In [3]:
rnaseq

,Ovary_GTEX-15ER7-2326-SM-7KUN3_sorted,Ovary_GTEX-1J8JJ-0826-SM-AHZ3F_sorted,Ovary_GTEX-11P81-1526-SM-5P9GS_sorted,Ovary_GTEX-1LGOU-1026-SM-DHXL4_sorted,Ovary_GTEX-1ICG6-1526-SM-C1YQT_sorted,Ovary_GTEX-WXYG-1426-SM-4ONCK_sorted,Breast_Mammary_Tissue_GTEX-1ICG6-1326-SM-ACKWR_sorted,Breast_Mammary_Tissue_GTEX-1JKYR-0926-SM-CMKGJ_sorted,Breast_Mammary_Tissue_GTEX-1K2DA-1726-SM-CGQGS_sorted,Breast_Mammary_Tissue_GTEX-UTHO-1026-SM-3GAF7_sorted,...,Prostate_GTEX-1IDJF-2126-SM-AHZ2S_sorted,Prostate_GTEX-1KANA-1026-SM-D3L9L_sorted,Prostate_GTEX-1KXAM-1126-SM-E9TJU_sorted,Prostate_GTEX-1B8KE-1026-SM-731EQ_sorted,Prostate_GTEX-132QS-1126-SM-5P9GC_sorted,Prostate_GTEX-1K2DA-1226-SM-CGQGH_sorted,Brain_Substantia_nigra_GTEX-QMR6-0011-R2A-SM-32PKV_sorted,Brain_Substantia_nigra_GTEX-X4XX-0011-R2A-SM-3P623_sorted,Brain_Substantia_nigra_GTEX-QDT8-0011-R2A-SM-32PKQ_sorted,Brain_Substantia_nigra_GTEX-15DYW-0011-R2b-SM-7KUKY_sorted
ENST00000631435.1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENST00000415118.1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENST00000434970.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENST00000448914.1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENST00000632684.1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENST00000578601.2,2.441740,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.467940,0.000000,...,1.118750,0.000000,0.000000,1.356660,1.491450,0.000000,0.000000,0.000000,0.000000,0.000000
ENST00000640823.1,0.826646,0.232625,1.135540,0.502132,0.398603,0.747745,1.045180,0.755881,0.833204,1.737480,...,0.256764,0.000000,0.129059,0.949193,0.569971,1.120280,0.909888,0.141163,0.540213,0.194993
ENST00000584105.1,0.000000,0.000000,0.000000,0.078792,0.081681,0.000000,0.000000,0.077611,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.081825,0.000000,0.077405,0.000000,0.000000,0.000000,0.000000
ENST00000592752.1,1.824500,1.242400,0.568687,1.521110,0.995720,1.197790,0.403738,0.942667,1.632090,0.434718,...,2.723730,0.616016,0.550226,1.424690,0.364994,1.462800,0.281095,0.150583,0.581012,0.233804


In [39]:
tss_protein = tss[tss.Gene_Type=="protein-coding"]

expression_mask=rnaseq.loc[:,rnaseq.columns.str.startswith('pt')] > 1
expressed = rnaseq[expression_mask.any(axis=1)]
expressed_protein = expressed[expressed.type=="protein-coding"]

(13259,)

In [49]:
tss_rnaseq_protein_intersect = np.intersect1d(expressed_protein.ens_gene.dropna().values, tss_protein.Nearest_Ensembl.dropna().values)
tss_rnaseq_protein_intersect.shape[0]/tss_protein.Nearest_Ensembl.dropna().unique().shape[0]

0.9612338788747266

In [50]:
tss_rnaseq_intersect = np.intersect1d(expressed.ens_gene.dropna().values, tss.Nearest_Ensembl.dropna().values).shape
tss_rnaseq_intersect[0]/tss.Nearest_Ensembl.dropna().unique().shape[0]

0.8513875285667646

In [62]:
both=tss_rnaseq_protein_intersect.shape[0] 
tss_only = tss_protein.Nearest_Ensembl.dropna().unique().shape[0] - both
rnaseq_only = expressed_protein.ens_gene.dropna().unique().shape[0] - both

c=venn2(subsets = (tss_only, rnaseq_only, both),alpha=0.7,set_colors=('blue','gray'),set_labels = ('5P-seq', 'RNAseq'))
plt.savefig('figureS5.pdf', bbox_inches="tight")
plt.close()

print(f'Total TSRs in protein-coding genes (multiple TSS per gene possible): {tss_protein.Nearest_Ensembl.dropna().unique().shape[0]}')
print(f'Protein-coding genes expressed (multiple transcripts per gene possible): {expressed_protein.ens_gene.dropna().unique().shape[0]}')
print(f'Genes for which both TRS and RNAseq evidence for expression:  {both} ({round(both/tss_protein.Nearest_Ensembl.dropna().unique().shape[0]*100,1)}%)')
print(f'Fraction of TRS: TRS only and shared with RNAseq evidence for expression:  {round(tss_only/tss_protein.Nearest_Ensembl.dropna().unique().shape[0]*100,1)}% and {round(both/tss_protein.Nearest_Ensembl.dropna().unique().shape[0]*100,1)}%')
print(f'Fraction of expressed genes: expressed only and shared with TRS evidence:  {round(rnaseq_only/expressed_protein.ens_gene.dropna().unique().shape[0]*100,1)}% and {round(both/expressed_protein.ens_gene.dropna().unique().shape[0]*100,1)}%')


Total TSRs in protein-coding genes (multiple TSS per gene possible): 13259
Protein-coding genes expressed (multiple transcripts per gene possible): 15770
Genes for which both TRS and RNAseq evidence for expression:  12745 (96.1%)
Fraction of TRS: TRS only and shared with RNAseq evidence for expression:  3.9% and 96.1%
Fraction of expressed genes: expressed only and shared with TRS evidence:  19.2% and 80.8%
